- 5장에서는 결과가 이산형 분류로 표현되는 분류모델을 다루었지만, 이번장에서는 목표변수( 종속 )변수가 실수값을 갖는 회귀모형( regression )모델을 다룸.
- 회귀모형도 분류모델과 같이 지도학습( supervised learning )에 속함.

### Regression models의 사용예
- 주식가격 및 경제지표 예측
- 부도에 의한 손실에 예측(  분류모델을 부도확률을 예측 )
- 추천( 사용자의 특성치를 고려함 )
- 행위와 소비패턴을 고려한 고객 생애 가치( Customer Lifetime Value, CLTV )를 예측


### 이번장의 내용
- MLlib에 있는 회귀모형에서 사용가능한 자료형태
- 특성 추출과 회귀모형을 위한 목표변수 변환
- MLlib을 사용해서 회귀모형을 훈련
- 훈련된 회귀모형을 이용해서 예측방법
- 교차 유효성 검사(cross-validation)를 이용해서 회귀모형의 여러가지 파라메타가 성능에 미치는 영향을 조사

# 1. Types of regression models 

## 1.1 Least squares regression

- L1 regularization은 오차의 절대값을 합
- L2 regularization은 오차의 제곱합을 ROOT 처리한 방식

- L1 regularization을 적용한 방식을  lasso regression
- L2 regularization을 적용한 방식을  ridge regression( 일반적인 방식 )


![행렬을 이용한 중회귀모형](regress_01.jpg "행렬을 이용한 중회귀모형")

![행렬을 이용한 중회귀모형](regress_02.jpg "행렬을 이용한 중회귀모형")

![행렬을 이용한 중회귀모형](regress_03.jpg "행렬을 이용한 중회귀모형")

![행렬을 이용한 중회귀모형](regress_04.jpg "행렬을 이용한 중회귀모형")

## 1.2 Decision trees for regression

- x 축이 input 변수,  y축이 목표변수
- 빨간색 선은 선형모델의 예측함수를 보여주고,  녹색선은 decision tree의 예측함수를 보여줌

![Linear model and decision tree prediction functions for regression](regress_05.jpg "Linear model and decision tree prediction functions for regression")

# 2. Extracting the right features from your data

- 분류모델과 같이 input features( 독립변수 )을 만들고, 실질적인 차이는 목표변수( 종속변수 )가 실수값임.
- MLlib에서는 LabeledPoint 클래스가 Double type으로 label 필드가 제공하고, 목표변수로 지정함.

## 2.1. Extracting features from the bike sharing dataset

- 도시 자전거 공유 시스템에서 발생한 자전거의 일별, 시간별 렌탈정보 
- 날짜와 시간, 날씨, 계절, 휴일정보가 포함
- http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset 에서 제공
-  wget http://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip 
- 파일 구성 : readme.txt, hour.csv, day.csv

### readme.txt의 내용
- 1) instant: This is the record ID
- 2) dteday: This is the raw date
- 3)  season: This is different seasons such as spring, summer, winter, and fall
- 4)  yr: This is the year (2011 or 2012)
- 5)  mnth: This is the month of the year
- 6)  hr: This is the hour of the day
- 7)  holiday: This is whether the day was a holiday or not
- 8)  weekday: This is the day of the week
- 9)  workingday: This is whether the day was a working day or not
- 10)  weathersit: This is a categorical variable that describes the weather at a particular time
- 11)  temp: This is the normalized temperature
- 12)  atemp: This is the normalized apparent temperature
- 13)  hum: This is the normalized humidity( 습도 ) 
- 14)  windspeed: This is the normalized wind speed( 바람세기 )
- 15)  cnt: This is the target variable, that is, the count of bike rentals for that hour

### hour.csv은 헤더에 컬럼명을 포함하고 있어서 헤더를 제거가 필요
- sed 1d hour.csv > hour_noheader.csv

### PySpark shell 시작하고 데이터 로딩과 보기

In [ ]:
$ MASTER=local[4]  bin/pyspark
path = "data/hour_noheader.csv"
raw_data = sc.textFile(path)
num_data = raw_data.count()
records = raw_data.map(lambda x: x.split(","))
first = records.first()
print first
print num_data

- record ID, raw date 컬럼은 무시
- 목표변수 cnt는 casual와 registered의 값의 합이기 때문에, casual와 registered 컬럼들도 무시
- 독립변수는 12개이고, 1 ~ 8 변수까지는 범주형( categorical)이고, 나머지 4개는 표준화된 실수값( 0.0 ~ 1.0 )
- records 에 저장된 데이터를 여러번 사용하기 위해서 캐싱 처리함

In [ ]:
records.cache()

- 범주형 변수 8개는 binary encoding 처리 필요
- get_mapping()함수는  범주형 변수를 Python의 dictionary변수로 추출하는 함수
- 2번째 범수를 추출해봄.

In [ ]:
def get_mapping(rdd, idx):
    return rdd.map(lambda fields: fields[idx]).distinct().zipWithIndex().collectAsMap()


print "Mapping of first categorical feasture column: %s" % get_mapping(records, 2)

- 각각의 모든 범주형 변수에 대해서 적용

In [ ]:
mappings = [get_mapping(records, i) for i in range(2,10)]
cat_len = sum(map(len, mappings))
num_len = len(records.first()[11:15])
total_len = num_len + cat_len


print "Feature vector length for categorical features: %d" % cat_len
print "Feature vector length for numerical features: %d" % num_len
print "Total feature vector length: %d" % total_len

## 2.2. Creating feature vectors for the linear model

- 범주형 값들을 모두 각각 지시변수로 만드는 과정과 연속형변수와 합쳐주는 함수 작성
- 선형대수(linear algebra) util LIB인 numpy를 사용함
- LabeledPoint 클래스에 목표변수와 독립변수들을 맵핑함.

In [ ]:
from pyspark.mllib.regression import LabeledPoint
import numpy as np

def extract_features(record):
    cat_vec = np.zeros(cat_len)
    i = 0
    step = 0
    for field in record[2:9]:
        m = mappings[i]
        idx = m[field]
        cat_vec[idx + step] = 1
        i = i + 1
        step = step + len(m)
    num_vec = np.array([float(field) for field in record[10:14]])
    return np.concatenate((cat_vec, num_vec))


def extract_label(record):
    return float(record[-1])

- extract_features()와 extract_label()을 이용해서 독립변수(특성치)을 뽑아내고 각각의 레코드에 대해서 lable을 생성
- 그리고, 결과 확인

In [ ]:
data = records.map(lambda r: LabeledPoint(extract_label(r), extract_features(r)))

first_point = data.first()
print "Raw data: " + str(first[2:])
print "Label: " + str(first_point.label)
print "Linear Model feature vector:\n" + str(first_point.features)
print "Linear Model feature vector length: " + str(len(first_point.features))

## 2.3. Creating feature vectors for the decision tree

In [ ]:
def extract_features_dt(record):
    return np.array(map(float, record[2:14]))

data_dt = records.map(lambda r: LabeledPoint(extract_label(r),
extract_features_dt(r)))
first_point_dt = data_dt.first()
print "Decision Tree feature vector: " + str(first_point_dt.features)
print "Decision Tree feature vector length: " +
str(len(first_point_dt.features))

# 3. Training and using regression models

- 이전에 만든 훈련데이터( [LabeledPoint] RDD )을 train()함수에 넣어주면 훈련이 가능함.
- SGD( Stochastic gradient descent , 확률적 기울기 하강 )은 함수의 극대값 또는 극소값을 구하기 위해 현재 위치에서 함수값의 변화가 가장 큰 방향으로 이동한다는 것
![확률적 기울기 하강](regress_06.jpg "확률적 기울기 하강")
![확률적 기울기 하강](regress_07.jpg "확률적 기울기 하강")

In [ ]:
from pyspark.mllib.regression import LinearRegressionWithSGD
from pyspark.mllib.tree import DecisionTree

help(LinearRegressionWithSGD.train)

help(DecisionTree.trainRegressor)


## 3.1 Training a regression model on the bike sharing dataset

In [ ]:
linear_model = LinearRegressionWithSGD.train(data, iterations=10, step=0.1, intercept=False) 

true_vs_predicted = data.map( lambda p: (p.label, linear_model.predict(p.features)) )
print "Linear Model predictions: " + str(true_vs_predicted.take(5) )

In [ ]:
dt_model = DecisionTree.trainRegressor(data_dt,{})
preds = dt_model.predict(data_dt.map(lambda p: p.features))
actual = data.map(lambda p: p.label)
true_vs_predicted_dt = actual.zip(preds)

print "Decision Tree predictions: " + str(true_vs_predicted_dt.take(5))
print "Decision Tree depth: " + str(dt_model.depth())
print "Decision Tree number of nodes: " + str(dt_model.numNodes())

# 4. Evaluating the performance of regression models

## 4.1. Mean Squared Error and Root Mean Squared Error

![MSE](regress_08.jpg "MSE")

In [ ]:
def squared_error(actual, pred):
    return (pred - actual)**2

## 4.2. Mean Absolute Error

![MAE](regress_09.jpg "MAE")

In [ ]:
def abs_error(actual, pred):
    return np.abs(pred - actual)

## 4.3. Root Mean Squared Log Error

- 목표변수의 범위가 넓을때 유용

## 4.4. The R-squared coefficient

- 결정 계수( coefficient of determination )라고 잘 알려짐.
- (전체변동 - 에러변동 ) / 전체변동  ==  모델에 의한 변동 / 전체변동
- 0 ~ 1 사이값을 갖으며, 1에 가까울수록 모델의 설명력이 높음.

In [ ]:
def squared_log_error(pred, actual):
    return (np.log(pred + 1) - np.log(actual + 1))**2

## 4.5. Computing performance metrics on the bike sharing dataset

### 4.5.1. Linear model

In [ ]:
mse = true_vs_predicted.map(lambda (t, p): squared_error(t, p)).mean()
mae = true_vs_predicted.map(lambda (t, p): abs_error(t, p)).mean()
rmsle = np.sqrt(true_vs_predicted.map(lambda (t, p): squared_log_error(t, p)).mean())
print "Linear Model - Mean Squared Error: %2.4f" % mse
print "Linear Model - Mean Absolute Error: %2.4f" % mae
print "Linear Model - Root Mean Squared Log Error: %2.4f" % rmsle

### 4.5.2. Decision tree

In [ ]:
mse_dt = true_vs_predicted_dt.map(lambda (t, p): squared_error(t, p)).mean()
mae_dt = true_vs_predicted_dt.map(lambda (t, p): abs_error(t, p)).mean()
rmsle_dt = np.sqrt(true_vs_predicted_dt.map(lambda (t, p): squared_log_error(t, p)).mean())
print "Decision Tree - Mean Squared Error: %2.4f" % mse_dt
print "Decision Tree - Mean Absolute Error: %2.4f" % mae_dt
print "Decision Tree - Root Mean Squared Log Error: %2.4f" %rmsle_dt

# 5. Improving model performance and tuning parameters

## 5.1 Transforming the target variable